In [20]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, callbacks

In [21]:
# Load the pre-trained VGG16 model without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))


In [22]:
# Freeze the base model layers
base_model.trainable = False

In [23]:
# Create the CNN model by adding the base model and additional layers
model = tf.keras.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [24]:
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [25]:
# Load and preprocess your dataset
train_data_dir = 'Dataset/Training'
validation_data_dir = 'Dataset/Validation'

In [26]:
# Define the image size and batch size
image_size = (64, 64)
batch_size = 32

In [27]:
# Data preprocessing and augmentation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,          # Normalize pixel values to [0, 1]
    shear_range=0.2,         # Shear transformations
    zoom_range=0.2,          # Zoom transformations
    horizontal_flip=True)   # Horizontal flips

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [28]:
# Create data generators for training and validation sets
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary')

Found 35 images belonging to 2 classes.
Found 7 images belonging to 2 classes.


In [35]:

# Define callbacks
early_stopping = callbacks.EarlyStopping(
    monitor='accuracy',
    patience=40,
    restore_best_weights=True
)

model_checkpoint = callbacks.ModelCheckpoint(
    'fcd_model.h5',
    monitor='val_accuracy',
    save_best_only=True
)

In [36]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/100
1/1 [==============================] - 0s 142ms/step - loss: 0.1640 - accuracy: 1.0000
Epoch 2/100
1/1 [==============================] - 0s 62ms/step - loss: 0.0570 - accuracy: 1.0000
Epoch 3/100
1/1 [==============================] - 0s 351ms/step - loss: 0.1740 - accuracy: 0.9375
Epoch 4/100
1/1 [==============================] - 0s 60ms/step - loss: 0.4568 - accuracy: 0.6667
Epoch 5/100
1/1 [==============================] - 0s 357ms/step - loss: 0.1741 - accuracy: 0.9688
Epoch 6/100
1/1 [==============================] - 0s 59ms/step - loss: 0.2081 - accuracy: 1.0000
Epoch 7/100
1/1 [==============================] - 0s 63ms/step - loss: 0.0516 - accuracy: 1.0000
Epoch 8/100
1/1 [==============================] - 0s 67ms/step - loss: 0.2922 - accuracy: 1.0000
Epoch 9/100
1/1 [==============================] - 0s 68ms/step - loss: 0.0372 - accuracy: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 359ms/step - loss: 0.1445 - accuracy: 1.0000
Epoch 11/100
1/

1/1 [==============================] - 0s 61ms/step - loss: 0.1128 - accuracy: 1.0000
Epoch 33/100
1/1 [==============================] - 0s 354ms/step - loss: 0.1095 - accuracy: 1.0000
Epoch 34/100
1/1 [==============================] - 0s 349ms/step - loss: 0.1842 - accuracy: 0.9375
Epoch 35/100
1/1 [==============================] - 0s 370ms/step - loss: 0.1111 - accuracy: 1.0000
Epoch 36/100
1/1 [==============================] - 0s 375ms/step - loss: 0.1365 - accuracy: 1.0000
Epoch 37/100
1/1 [==============================] - 0s 101ms/step - loss: 0.0544 - accuracy: 1.0000
Epoch 38/100
1/1 [==============================] - 0s 61ms/step - loss: 0.1094 - accuracy: 1.0000
Epoch 39/100
1/1 [==============================] - 0s 362ms/step - loss: 0.1185 - accuracy: 0.9688
Epoch 40/100
1/1 [==============================] - 0s 60ms/step - loss: 0.0765 - accuracy: 1.0000
Epoch 41/100
1/1 [==============================] - 0s 366ms/step - loss: 0.1069 - accuracy: 1.0000


In [37]:
model.save('fcd_final.h5')